In [1]:
import pandas as pd
import numpy as np
import fuzzywuzzy
import matplotlib.pyplot as plt
import seaborn as sns
import re
import bs4 
import warnings
warnings.filterwarnings('ignore')

In [2]:
df=pd.read_csv("trainnn.csv")

In [3]:
df=df.sample(100000, random_state=1)


In [4]:
df.shape

(100000, 6)

In [14]:
def preprocess(text):
    text = str(text).lower().strip()
    # special symbols
    text=text.replace("%", " percent") 
    text=text.replace("$", " dollar")
    text = text.replace("₹", " ruppee")
    text = text.replace("€", " euro")
    text = text.replace("@", " at")
    # occurs 900 times
    text=text.replace("[math]","")
    
    
    text = text.replace(',000,000,000 ', 'b ')
    text = text.replace(',000,000 ', 'm ')
    text = text.replace(',000 ', 'k ')
    text = re.sub(r'([0-9]+)000000000', r'\1b', text)
    text = re.sub(r'([0-9]+)000000', r'\1m', text)
    text = re.sub(r'([0-9]+)000', r'\1k', text)
    
    contractions = { 
    "ain't": "am not",
    "aren't": "are not",
    "can't": "can not",
    "can't've": "can not have",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he would",
    "he'd've": "he would have",
    "he'll": "he will",
    "he'll've": "he will have",
    "he's": "he is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how is",
    "i'd": "i would",
    "i'd've": "i would have",
    "i'll": "i will",
    "i'll've": "i will have",
    "i'm": "i am",
    "i've": "i have",
    "isn't": "is not",
    "it'd": "it would",
    "it'd've": "it would have",
    "it'll": "it will",
    "it'll've": "it will have",
    "it's": "it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she would",
    "she'd've": "she would have",
    "she'll": "she will",
    "she'll've": "she will have",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so as",
    "that'd": "that would",
    "that'd've": "that would have",
    "that's": "that is",
    "there'd": "there would",
    "there'd've": "there would have",
    "there's": "there is",
    "they'd": "they would",
    "they'd've": "they would have",
    "they'll": "they will",
    "they'll've": "they will have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we would",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what will",
    "what'll've": "what will have",
    "what're": "what are",
    "what's": "what is",
    "what've": "what have",
    "when's": "when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where is",
    "where've": "where have",
    "who'll": "who will",
    "who'll've": "who will have",
    "who's": "who is",
    "who've": "who have",
    "why's": "why is",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you would",
    "you'd've": "you would have",
    "you'll": "you will",
    "you'll've": "you will have",
    "you're": "you are",
    "you've": "you have"
    }
    text_decontracted = []

    for word in text.split():
        if word in contractions:
            word = contractions[word]

        text_decontracted.append(word)
    text = ' '.join(text_decontracted)
    text = text.replace("'ve", " have")
    text = text.replace("n't", " not")
    text = text.replace("'re", " are")
    text = text.replace("'ll", " will")
    
    # Removing HTML tags
    text = bs4.BeautifulSoup(text)
    text = text.get_text()
    
    # Remove punctuations
    pattern = re.compile('\W')
    text = re.sub(pattern, ' ', text).strip()
    
    return text

In [15]:
df["question1"] = df["question1"].apply(preprocess)
df["question2"] = df["question2"].apply(preprocess)

In [16]:
df["question1"].sample(5)

190403    can a cs degree from imperial college london g...
174583    what are the best sports showrooms in coimbato...
162665     what are some major events that happened in 1994
382881    should not modi government privatize railways ...
190877    which countries have names that start with the...
Name: question1, dtype: object

In [17]:
df.sample(5)

,id,qid1,qid2,question1,question2,is_duplicate
124578,124578,201138,121542,why half filled orbital is stable than partial...,why are half filled and fully filled orbitals ...,0
192664,192664,169481,292356,what is the best way to calculate my gross income,how can i calculate gross in gujarat government,1
288437,288437,305905,409397,what is demography,how is demography studied,0
178333,178333,273923,273924,what is market analysis,what is marketing analysis,1
221495,221495,42298,53499,what was the significance of the battle of som...,what was the significance of the battle of som...,1


In [19]:
df["q1_num_words"] = df["question1"].apply(lambda x: len(x.split()))
df["q2_num_words"] = df["question2"].apply(lambda x: len(x.split()))

In [20]:
df["q1_len"]= df["question1"].str.len()
df["q2_len"]= df["question2"].str.len()

In [21]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate,q1_len,q2_len,q1_num_words,q2_num_words
237030,237030,33086,348102,how can i stop playing video games,should i stop playing video games with my child,0,34,47,7,9
247341,247341,73272,8624,who is better donald trump or hillary clinton,why is hillary clinton a better choice than do...,1,45,56,8,10
246425,246425,359482,359483,what do you think is the chance that sometime ...,do you think there will be another world war n...,1,137,76,28,15
306985,306985,1357,47020,why are so many questions posted to quora that...,why do people write questions on quora that co...,1,85,85,16,16
225863,225863,334315,334316,can there even be a movie ever rated 10 10 on ...,what are your 10 10 movies,0,50,26,12,6


In [22]:
def common_word(text):
    w1=set(map(lambda word:word.lower().strip(),text["question1"].split()))
    w2=set(map(lambda word:word.lower().strip(),text["question2"].split()))
    return len(w1 & w2)

In [23]:
df["common_word"] = df.apply(common_word, axis=1)

In [24]:
def total_word(text):
    w1=set(map(lambda word:word.lower().strip(),text["question1"].split()))
    w2=set(map(lambda word:word.lower().strip(),text["question2"].split()))
    return len(w1)+len(w2)

In [25]:
df["total_word"] = df.apply(total_word, axis=1)

In [30]:
df["word_share"]=round(df["common_word"]/df["total_word"],2)

In [31]:
df.columns

Index(['id', 'qid1', 'qid2', 'question1', 'question2', 'is_duplicate',
       'q1_len', 'q2_len', 'q1_num_words', 'q2_num_words', 'common_word',
       'total_word', 'word_share'],
      dtype='object')

In [32]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate,q1_len,q2_len,q1_num_words,q2_num_words,common_word,total_word,word_share
237030,237030,33086,348102,how can i stop playing video games,should i stop playing video games with my child,0,34,47,7,9,5,16,0.31
247341,247341,73272,8624,who is better donald trump or hillary clinton,why is hillary clinton a better choice than do...,1,45,56,8,10,6,18,0.33
246425,246425,359482,359483,what do you think is the chance that sometime ...,do you think there will be another world war n...,1,137,76,28,15,13,39,0.33
306985,306985,1357,47020,why are so many questions posted to quora that...,why do people write questions on quora that co...,1,85,85,16,16,5,30,0.17
225863,225863,334315,334316,can there even be a movie ever rated 10 10 on ...,what are your 10 10 movies,0,50,26,12,6,1,16,0.06


# Advance Features

In [40]:
from nltk.corpus import stopwords
def token_features(text):
    q1=text["question1"]
    q2=text["question2"]
    
    SAFE_DIV=0.0001
    
    token_features = [0,0]*8
    STOP_WORDS=stopwords.words("english")
    
    q1_tokens=q1.split()
    q2_tokens=q2.split()
    
    if len(q1_tokens)==0 or len(q2_tokens)==0:
        return token_features
    
    q1_words=set([word for word in q1_tokens if word not in STOP_WORDS])
    q2_words=set([word for word in q2_tokens if word not in STOP_WORDS])
    
    q1_stopwords=set([word for word in q1_tokens if word in STOP_WORDS])
    q2_stopwords=set([word for word in q2_tokens if word in STOP_WORDS])
    
    
    common_words_count=len(q1_words.intersection(q2_words))
    common_stopwords_count=len(q1_stopwords.intersection(q2_stopwords))
    
    common_token_count=len(set(q1_tokens).intersection(set(q2_tokens)))
    
    token_features[0]=common_words_count/(min(len(q1_words),len(q1_words))+SAFE_DIV)
    token_features[1]=common_words_count/(max(len(q1_words),len(q2_words))+SAFE_DIV)
    token_features[2]=common_stopwords_count/(min(len(q1_stopwords),len(q2_stopwords))+SAFE_DIV)
    token_features[3]=common_stopwords_count/(max(len(q1_stopwords),len(q2_stopwords))+SAFE_DIV)
    token_features[4]=common_token_count/(min(len(q1_tokens),len(q2_tokens))+SAFE_DIV)
    token_features[5]=common_token_count/(max(len(q1_tokens),len(q2_tokens))+SAFE_DIV)
    
    
    token_features[6]=int(q1_tokens[-1]==q2_tokens[-1])
    token_features[7]=int(q1_tokens[0]==q2_tokens[0])
    return token_features

In [41]:
tk_features=df.apply(token_features, axis=1)


df["cwc_min"]=list(map(lambda x:x[0]    ,tk_features))
df["cwc_max"]=list(map(lambda x:x[1]    ,tk_features))
df["csc_min"]=list(map(lambda x:x[2]    ,tk_features))
df["csc_max"]=list(map(lambda x:x[3]    ,tk_features))
df["ctc_min"]=list(map(lambda x:x[4]    ,tk_features)) 
df["ctc_max"]=list(map(lambda x:x[5]    ,tk_features))
df["last_word_eq"]=list(map(lambda x:x[6]    ,tk_features))
df["first_word_eq"]=list(map(lambda x:x[7]    ,tk_features))

In [44]:
import distance
def fetch_length_features(text):
    q1=text["question1"]
    q2=text["question2"]
    
    length_features=[0]*3
    
    q1_tokens=q1.split()
    q2_tokens=q2.split()
    
    if len(q1)==0 or len(q2)==0:
        return length_features
    
    length_features[0]=abs(len(q1)-len(q2))
    length_features[1]=(len(q1_tokens)+len(q1_tokens))/2
    strs=list(distance.lcsubstrings(q1,q2))
    length_features[2]=len(strs[0])/(min(len(q1),len(q2))+1)
    
    return length_features

In [45]:
length_features=df.apply(fetch_length_features, axis=1)

df["abs_len_diff"]=list(map(lambda x:x[0]    ,length_features))
df["mean_len"]=list(map(lambda x:x[1]    ,length_features))
df["longest_substr_ratio"]=list(map(lambda x:x[2]    ,length_features))

In [46]:
df.columns

Index(['id', 'qid1', 'qid2', 'question1', 'question2', 'is_duplicate',
       'q1_len', 'q2_len', 'q1_num_words', 'q2_num_words', 'common_word',
       'total_word', 'word_share', 'cwc_min', 'cwc_max', 'csc_min', 'csc_max',
       'ctc_min', 'ctc_max', 'last_word_eq', 'first_word_eq', 'abs_len_diff',
       'mean_len', 'longest_substr_ratio'],
      dtype='object')

In [47]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate,q1_len,q2_len,q1_num_words,q2_num_words,...,cwc_max,csc_min,csc_max,ctc_min,ctc_max,last_word_eq,first_word_eq,abs_len_diff,mean_len,longest_substr_ratio
237030,237030,33086,348102,how can i stop playing video games,should i stop playing video games with my child,0,34,47,7,9,...,0.799984,0.333322,0.249994,0.714276,0.555549,0,0,13,7.0,0.771429
247341,247341,73272,8624,who is better donald trump or hillary clinton,why is hillary clinton a better choice than do...,1,45,56,8,10,...,0.833319,0.333322,0.249994,0.749991,0.599994,0,0,11,8.0,0.347826
246425,246425,359482,359483,what do you think is the chance that sometime ...,do you think there will be another world war n...,1,137,76,28,15,...,0.499996,0.999986,0.538457,0.866661,0.464284,0,0,61,28.0,0.298701
306985,306985,1357,47020,why are so many questions posted to quora that...,why do people write questions on quora that co...,1,85,85,16,16,...,0.333330,0.333328,0.285710,0.312498,0.312498,0,1,0,16.0,0.139535
225863,225863,334315,334316,can there even be a movie ever rated 10 10 on ...,what are your 10 10 movies,0,50,26,12,6,...,0.166664,0.000000,0.000000,0.166664,0.083333,0,0,24,12.0,0.259259


In [48]:
from fuzzywuzzy import fuzz

def fetch_fuzzy_ratio(text):
    q1=text["question1"]
    q2=text["question2"]
    
    fuzzy_features=[0]*4
    
    fuzzy_features[0]=fuzz.ratio(q1,q2)
    fuzzy_features[1]=fuzz.partial_ratio(q1,q2)
    fuzzy_features[2]=fuzz.token_sort_ratio(q1,q2)
    fuzzy_features[3]=fuzz.token_set_ratio(q1,q2)
    return fuzzy_features
    
    

In [49]:
fuzzy_features=df.apply(fetch_fuzzy_ratio, axis=1)
df["fuzzy_ratio"]=list(map(lambda x:x[0]    ,fuzzy_features))
df["partial_ratio"]=list(map(lambda x:x[1]    ,fuzzy_features))
df["token_sort_ratio"]=list(map(lambda x:x[2]    ,fuzzy_features))
df["token_set_ratio"]=list(map(lambda x:x[3]    ,fuzzy_features))